In [1]:
from Recordings import AEPFeedbackRecording, Recordings, Recording

In [2]:
# Load the data
basepath = './Recordings'
recordings = Recordings(basepath)
recordings.print_info(group='*', session='*', subject_id='*', experiment_id='aep_feedback')

Reading recording: ./Recordings/1/.DS_Store
Reading recording: ./Recordings/1/7/1/7_aep_2024-03-28_14-34-11_1.xdf
Reading recording: ./Recordings/1/7/1/7_aep_feedback_2024-03-28_14-41-11_1.xdf
Creating RawArray with float64 data, n_channels=8, n_times=108836
    Range : 0 ... 108835 =      0.000 ...   435.340 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 1651 samples (6.604 s)
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 60 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window wit

RuntimeError: Event time samples were not unique. Consider setting the `event_repeated` parameter."

In [ ]:
aep_feedback_recordings = recordings.filter_by(group='*', session='*', subject_id='*', experiment_id='aep_feedback')

In [ ]:
test_recording = aep_feedback_recordings[0]

In [ ]:
# trial-begin
# standard/oddball
# response-received-(arrow_up/arrow_down)/response-was-missed
# response-was-(correct/incorrect)/None
# rt-(XXXms)
# trial-end
# ---

In [ ]:
test_recording._epochs
test_recording._raw.plot(
    events=events,
    start=5,
    duration=10,
    color="gray",
    event_color={1: "r", 2: "g", 3: "b", 4: "m", 5: "y", 32: "k"},
)